# Recuperação da Informação 2018.1
### Aluna: Hadrizia Santos

## Lab01 - Parte 1 - Construção de Índice Invertido e Busca Booleana

Neste lab, inicialmente será criado um índice invertido. Depois, serão implementados os algoritmos de busca booleana: 
* 1-termo: usuário passa um termo e o algoritmo retorna os documentos associados;
* Busca AND: todos os termos da consulta devem estar presentes na página;
* Busca OR: algum dos termos da consulta deve estar presente na página.

### 1. Importando bibliotecas e os dados a serem indexados
   
   Inicialmente importam-se as bibliotecas, declara-se as variáveis a serem utilizadas e realiza-se um pré-processamento nos dados.

In [4]:
import pandas as pd
from collections import defaultdict
import operator

dictionary = defaultdict(list)
dic_frequency = {}

FILE_NAME = 'noticias_estadao.csv'

df = pd.read_csv(FILE_NAME)

# criação de uma nova coluna para a junção do título da notícia com seu conteúdo
df['noticia'] = df.titulo + ' ' + df.conteudo

AND = 'and'
OR = 'or'

### 2. Implementando as funções a serem utilizadas posteriormente

   A seguir, foram implementadas algumas funções úteis para a criação do índice invertido: *create_tokens*, uma função que, dado um texto, retorna um array de palavras (tokens); *create_indexes*, que recebe como parâmetros uma lista de tokens e a identificação da notícia que possui estas palavras e adiciona, para cada palavra, a notícia em que ela aparece; *create_frequency*, que adiciona, para cada palavra, o número de frequência que ela aparece nas notícias armazenadas.

In [5]:
def create_tokens(text, docId):
    text = text.lower()
    tokens = text.split(' ')
    return tokens

def create_indexes(tokens, docId):
    for word in tokens:
        if docId not in dictionary[word]:
            dictionary[word].append(docId)
            
def create_frequency():
    for word in dictionary:
        dic_frequency[word] = len(dictionary[word])

### 3. Gerando tokens, criando índices para os dados e ordenando os documentos por id

   O laço abaixo percorre os dados, criando os tokens e os índices para cada notícia armazenada. Depois de criados e armazenados no dicionário *dictionary*, as frequências de cada palavra são armazenadas no dicionário *dic_frequency*.

In [6]:
for index, row in df.iterrows():
    token = create_tokens(row.noticia, row.idNoticia)
    create_indexes(token, row.idNoticia)
    
create_frequency()

### 4. Implementando os algoritmos de busca booleana

Os algoritmos implementados foram: *_one_term*, que recebe uma palavra e retorna os ids das notícias em que essa palavra aparece; *_and_search*, que recebe duas palavras e retorna os ids das notícias em que ambas aparecem; *_or_search*, que recebe duas palavras e retorna os ids das notícias em que pelo menos uma das palavras aparece; e *_generic_and_search*, uma função genérica que recebe N palavras e retorna os ids das notícias em que todas as palavras aparecem. 

A função *search* recebe uma busca, a separa em palavras e operandos e chama o algoritmo correspondente. 

In [7]:
def _one_term(element):
    if element in dictionary:
        return list(dictionary[element])
    else:
        return 0
    
def _and_search(first_element, second_element):
    return list(set(_one_term(second_element)).intersection(set(_one_term(first_element))))

def _or_search(first_element, second_element):
    return list(set(_one_term(second_element)).union(set(_one_term(first_element))))

# Questão BÔNUS
def _generic_and_search(terms_list):
    terms = sorted(terms_list, key = dic_frequency.get)
    result = dictionary[terms[0]]
    
    while len(terms) > 0 and result != 0:
        result = set(result).intersection(set(dictionary[terms[0]]))
        terms = terms[1:]
    return list(result)

def search(search):
    search = search.lower().split(' ')
    if len(search) == 1:
        return _one_term(search[0])
    elif len(search) == 3:
        first_element= search[0]
        operator = search[1]
        second_element = search[2]
        if operator == AND:
            return _and_search(first_element, second_element)
        elif operator == OR:
            return _or_search(first_element,second_element)
    elif len(search) > 3:
        terms_list = []
        for element in search:
            if element != AND:
                terms_list.append(element)
        return _generic_and_search(terms_list)
    else:
        raise ValueError('search must have at least a word.')
    

### 5. Adicionando algumas validações e exibindo resultados

   Abaixo foram adicionadas algumas validações dos algoritmos implementados acima:
   

In [8]:
#Validação de um termo:
assert len(search("candidatos")) == 1395

#Validação OR de dois termos:
assert len(search("debate OR presidencial")) == 1770
assert len(search("presidenciáveis OR corruptos")) == 164
assert len(search("Belo OR Horizonte")) == 331

#Validação AND de dois termos:
assert len(search("debate AND presidencial")) == 201
assert len(search("presidenciáveis AND corruptos")) == 0
assert len(search("Belo AND Horizonte")) == 242

#Validação para questão bônus - AND de N termos:
assert len(search("Campina AND Grande AND Paraíba AND Lula")) == 2
assert len(search("corrupto AND candidato AND propina")) == 1
assert len(search("corrupto AND PT")) == 20


In [12]:
# Usando a função edit_distance

def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

def edit_distance(search):
    distances = {}
    for word in dictionary:
        distances[word] = levenshteinDistance(search, word)
        
    sorted_x = sorted(distances.items(), key=operator.itemgetter(1))
    return sorted_x[0]



In [19]:
edit_distance('frafafra')

('rafaela', 3)

In [106]:
print(search('candidatos'))

[51, 73, 77, 84, 123, 127, 142, 143, 154, 161, 167, 189, 191, 263, 268, 276, 311, 343, 374, 375, 377, 383, 399, 417, 418, 444, 483, 484, 604, 605, 616, 623, 628, 632, 633, 640, 647, 651, 662, 688, 717, 718, 746, 747, 752, 761, 764, 777, 783, 790, 791, 792, 793, 795, 796, 801, 804, 813, 825, 827, 828, 829, 844, 859, 870, 871, 876, 890, 982, 985, 1005, 1011, 1014, 1016, 1019, 1020, 1034, 1040, 1050, 1053, 1058, 1059, 1068, 1075, 1077, 1086, 1091, 1096, 1097, 1099, 1103, 1106, 1108, 1112, 1113, 1115, 1116, 1117, 1119, 1123, 1124, 1128, 1131, 1134, 1140, 1143, 1148, 1158, 1162, 1163, 1165, 1166, 1168, 1170, 1172, 1174, 1175, 1178, 1186, 1189, 1191, 1193, 1194, 1195, 1197, 1198, 1199, 1200, 1201, 1202, 1203, 1208, 1209, 1210, 1213, 1214, 1218, 1228, 1229, 1230, 1234, 1236, 1237, 1244, 1248, 1251, 1257, 1265, 1267, 1269, 1274, 1277, 1278, 1292, 1293, 1295, 1306, 1307, 1310, 1315, 1317, 1324, 1327, 1330, 1337, 1338, 1339, 1343, 1344, 1348, 1349, 1355, 1356, 1359, 1364, 1371, 1372, 1374, 1380,

In [107]:
print(search('Campina AND Grande'))

[1952, 4802, 1987, 5382, 6694, 1770, 2763, 1068, 5870, 2777, 1370, 2779]


In [109]:
print(search('debate AND presidencial'))

[2054, 4615, 3592, 4619, 2574, 6670, 1038, 1043, 2068, 2069, 24, 4635, 4636, 3611, 1058, 1571, 4134, 7208, 6185, 2092, 1069, 4654, 2608, 7217, 1586, 1588, 3127, 6713, 6202, 2107, 2108, 5181, 7232, 1600, 1102, 1111, 4706, 3689, 1131, 1132, 2669, 1647, 1138, 1140, 5748, 1658, 637, 1151, 2176, 1155, 1158, 3206, 2184, 1672, 2705, 3219, 2711, 1180, 2211, 3235, 4261, 166, 4775, 3236, 3242, 1197, 2224, 5299, 3251, 3252, 6331, 5307, 703, 704, 3268, 713, 4814, 2255, 3792, 3793, 722, 2774, 4823, 2266, 5338, 3805, 4318, 3806, 3807, 3811, 1767, 6377, 234, 6892, 3830, 3322, 255, 7425, 3844, 776, 2313, 779, 6412, 1291, 6931, 1811, 4885, 3860, 1816, 1313, 2338, 1315, 3874, 805, 2853, 3367, 1320, 1325, 814, 1326, 3887, 4913, 5937, 1844, 1845, 5941, 5942, 6969, 1339, 1855, 2880, 5951, 6978, 1348, 327, 1864, 6987, 3916, 7505, 1362, 1873, 2388, 1367, 4442, 7004, 1374, 2911, 1379, 5479, 5480, 4460, 5996, 1391, 1394, 883, 1399, 6521, 3450, 6523, 1404, 1406, 3454, 6015, 6017, 4994, 3469, 2449, 1939, 4504, 4

In [110]:
print(search('debate OR presidencial'))

[1, 2, 4099, 4103, 4109, 16, 4113, 4119, 24, 4126, 4130, 4131, 4134, 46, 50, 4155, 61, 4158, 4161, 4166, 4169, 4170, 79, 86, 88, 4197, 4198, 105, 107, 109, 4207, 4215, 4216, 122, 4223, 130, 4234, 4236, 140, 143, 4243, 4246, 158, 4261, 166, 165, 4265, 4267, 4274, 4276, 182, 4279, 4278, 189, 4286, 4290, 4294, 199, 201, 203, 204, 205, 209, 213, 4316, 4318, 4319, 228, 234, 235, 4331, 238, 240, 241, 244, 4341, 250, 4348, 4350, 255, 4354, 4355, 259, 261, 260, 4360, 4362, 4364, 273, 4373, 278, 4383, 295, 4395, 4399, 311, 4407, 4408, 4411, 4413, 4415, 4416, 4418, 327, 4427, 334, 336, 4433, 4437, 343, 345, 4442, 348, 353, 355, 357, 363, 4459, 4460, 368, 369, 4465, 4466, 4468, 4467, 374, 4470, 4474, 378, 383, 4479, 4480, 4484, 389, 392, 396, 399, 400, 4504, 4505, 4508, 413, 4511, 416, 417, 4513, 419, 4516, 4517, 4518, 4520, 4521, 4525, 4530, 4547, 452, 453, 4548, 4549, 4555, 461, 4558, 463, 4563, 4564, 471, 472, 475, 478, 484, 487, 4584, 491, 4589, 4593, 4606, 4611, 4615, 4617, 4619, 4620, 4625,

In [113]:
print(search('presidenciáveis AND corruptos'))

[]


In [114]:
print(search('presidenciáveis OR corruptos'))

[1537, 5121, 2051, 523, 2571, 5133, 2574, 2068, 2069, 3607, 538, 539, 5659, 2080, 3616, 2088, 4137, 2093, 2609, 3633, 4660, 3641, 68, 2628, 4677, 4170, 2123, 2125, 3664, 4177, 3666, 5713, 1109, 4184, 93, 2144, 2660, 3684, 2149, 1639, 4199, 2152, 1129, 2669, 3182, 2672, 2676, 3188, 2678, 5237, 4219, 126, 6783, 2686, 6785, 1158, 4743, 4235, 3217, 3730, 149, 5271, 6809, 1693, 6813, 160, 4258, 3747, 4260, 3243, 2732, 1198, 176, 3248, 2225, 180, 2740, 3767, 3260, 4294, 2248, 7369, 3786, 2764, 2253, 2285, 4847, 2813, 6910, 3843, 5895, 777, 272, 7441, 5906, 6931, 3860, 789, 2860, 1325, 1326, 304, 3377, 1847, 1851, 4926, 1343, 3391, 1859, 3397, 841, 330, 5472, 4965, 874, 3444, 3446, 3447, 375, 3962, 893, 4488, 2442, 3466, 4492, 1428, 3479, 2458, 6043, 3489, 422, 2471, 426, 430, 1461, 4025, 1470, 6079, 4034, 456, 1481, 4042, 2507, 7630, 1487, 5587, 2516, 7635, 7641, 3546, 7642, 6112, 3042, 6115, 3046, 2023, 2024, 2026, 2028, 2030, 497, 6131, 5110, 5115]


In [115]:
print(search('Belo AND Horizonte'))

[4, 1033, 3593, 1020, 13, 2061, 2063, 2064, 2065, 6160, 4115, 3604, 4116, 1046, 2074, 6174, 5665, 4646, 4649, 3626, 2044, 2095, 48, 3119, 2098, 2101, 1081, 3129, 1086, 1087, 1088, 1598, 1090, 1603, 3135, 1605, 1094, 3138, 3142, 3143, 74, 75, 5702, 5706, 2127, 4177, 3069, 5205, 6231, 3160, 4184, 5208, 6240, 2145, 4710, 1133, 3183, 624, 2672, 2673, 3186, 2164, 3188, 1142, 3698, 3700, 4723, 5242, 634, 4230, 4234, 6794, 4241, 4245, 3224, 4251, 1180, 4765, 1186, 4771, 2213, 6312, 2219, 4780, 6316, 4273, 4787, 4797, 3262, 4720, 1734, 4294, 206, 1230, 6354, 1242, 1762, 5133, 7396, 4333, 2798, 1264, 6897, 4850, 3316, 6900, 1270, 3326, 1280, 1797, 1798, 4357, 4361, 3852, 3860, 4375, 7451, 4893, 7453, 1823, 5407, 5919, 5410, 1315, 7460, 7463, 5417, 4398, 1329, 5425, 1331, 5946, 1343, 1858, 1861, 4937, 3915, 332, 2381, 4939, 4941, 1872, 1362, 3923, 1877, 3413, 1367, 1880, 3926, 4437, 1883, 2395, 2909, 4957, 4958, 5975, 5976, 5977, 7519, 1892, 1381, 4454, 1386, 1388, 1389, 6512, 1909, 3445, 7032, 

In [116]:
print(search('Belo OR Horizonte'))

[4, 4100, 1033, 13, 2061, 2063, 2064, 2065, 6160, 4115, 1044, 4116, 1046, 2074, 6174, 3109, 3110, 2044, 2095, 48, 3119, 2098, 2101, 1081, 3129, 1086, 1087, 1088, 3135, 1090, 3138, 1094, 3142, 3143, 74, 75, 2127, 4177, 5205, 6231, 3160, 4184, 5208, 6240, 2145, 3170, 1133, 3183, 4209, 3186, 2164, 3188, 1142, 5242, 5247, 5250, 4230, 4234, 3214, 4241, 4245, 3224, 3225, 4251, 1180, 2207, 1183, 1186, 2213, 6312, 2219, 6140, 6316, 6317, 3248, 4273, 2233, 3262, 196, 4294, 206, 1230, 6354, 211, 3284, 3288, 1242, 5341, 5133, 7396, 4333, 1264, 3316, 5364, 1270, 3326, 1280, 4357, 4361, 4372, 4375, 7451, 7453, 5407, 5410, 1315, 7460, 1318, 7463, 5417, 4398, 304, 1329, 5425, 1331, 2362, 2363, 2365, 1343, 332, 2381, 5456, 1362, 3413, 4437, 1367, 2395, 7519, 5476, 1381, 4454, 5477, 6500, 1386, 1388, 1389, 6512, 3445, 374, 7545, 1405, 1406, 7552, 7554, 6535, 5514, 7566, 1424, 7569, 4498, 1431, 1434, 5535, 5541, 1445, 3506, 2483, 5559, 6587, 2494, 6591, 3522, 7619, 5574, 3528, 3531, 3534, 4559, 5583, 55